In [1]:
import cv2
import pandas as pd
import numpy as np
from skimage import io
from shapely.geometry import Point, Polygon

In [2]:
IMG_PATH = "data/yukon.tif"

In [3]:
# def convert_tif_to_png(input_path, output_path):

#         # Open the TIFF image using rasterio
#         with rasterio.open(input_path) as src:
#             # Read the image data (assuming it's in the form of a numpy array)
#             image_data = src.read()

#         # Extract the individual channels (B, G, R, NIR)
#         b, g, r, nir, sw1, sw2 = image_data

#         b = np.clip(b * 255.0, 0, 255).astype(np.uint8)
#         g = np.clip(g * 255.0, 0, 255).astype(np.uint8)
#         r = np.clip(r * 255.0, 0, 255).astype(np.uint8)

#         # Create an RGB image by merging the channels (discard NIR)
#         rgb_image_data = np.stack([r, g, b], axis=-1)

#         # Create a Pillow Image from the RGB data
#         rgb_image = Image.fromarray(rgb_image_data, 'RGB')

#         # Save the result as a PNG image
#         rgb_image.save(output_path, format='PNG')
#         print(f"Conversion successful. Image saved to {output_path}")

# # Get input and output paths from command-line arguments
# input_tif_path = "data/yukon.tif"
# output_png_path = "img_rgb.png"

# convert_tif_to_png(input_tif_path, output_png_path)


In [4]:
IMG_RGB_PATH = "img_rgb.png"

In [5]:
# Global variable to store marked points
marked_points = []

def mark_four_points(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image is loaded successfully
    if image is None:
        print("Error: Unable to load the image.")
        return

    # Create a window and set the callback function for mouse events
    cv2.namedWindow('Mark Four Points')
    cv2.setMouseCallback('Mark Four Points', mark_point_callback)

    # Create a copy of the original image to draw on
    display_image = image.copy()

    while True:
        # Display the image
        for point in marked_points:
            cv2.circle(display_image, point, 5, (0, 255, 0), -1)

        cv2.imshow('Mark Four Points', display_image)

        # Wait for a key event and check if 'ESC' is pressed
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break

    # Close the window
    cv2.destroyAllWindows()

    return marked_points

def mark_point_callback(event, x, y, flags, param):
    global marked_points
    if event == cv2.EVENT_LBUTTONDOWN:
        marked_points.append((x, y))

if __name__ == "__main__":
    # Specify the path to your image
    image_path = IMG_RGB_PATH

    # Call the function to mark four points on the image
    marked_points = mark_four_points(image_path)

    # Print the coordinates of the marked points
    print("Coordinates of the marked points:")
    for i, point in enumerate(marked_points):
        print(f"Point {i + 1}: {point}")


Coordinates of the marked points:
Point 1: (375, 222)
Point 2: (417, 272)
Point 3: (463, 247)
Point 4: (414, 204)


In [6]:
def is_point_inside_polygon(point, polygon):
    """
    Check if a point is inside a polygon.
    """
    return Point(point).within(polygon)

def extract_cells_inside_polygon(array, polygon_vertices):
    """
    Extract cells from a 2D array that lie inside the given polygon.
    """

    w, h, ch = array.shape
    polygon = Polygon(polygon_vertices)
    extracted_cells = []

    # Iterate through array cells
    for i in range(w):
        for j in range(h):
            cell_coords = (i, j)

            # Check if the cell is inside the polygon
            if is_point_inside_polygon(cell_coords, polygon):
                extracted_cells.append(array[i, j, :])

    return extracted_cells

# Example usage
array  = io.imread(IMG_PATH)
polygon_vertices = marked_points

result = extract_cells_inside_polygon(array, polygon_vertices)
print("Extracted cells inside the polygon:", len(result))

Extracted cells inside the polygon: 2952


In [7]:
band_names = ["Red", 'Green', 'Blue', 'NIR', "SWIR_1", "SWIR_2"]

In [8]:
pd.DataFrame(result, columns=band_names).to_csv("land_test.csv")